In [1]:
from dataloader import *

In [2]:
import json
config_data = json.load(open('default.json', 'r'))

In [3]:
vocab, train_data_loader, val_data_loader, test_data_loader = get_datasets(config_data)

Using the saved vocab.


/opt/conda/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [4]:
train_data_loader

In [5]:
for i, (images, ingt, title, ing, ins, img_id) in enumerate(train_data_loader):
    break

RuntimeError: stack expects each tensor to be equal size, but got [3162] at entry 0 and [3160] at entry 3

In [ ]:
images.shape

In [ ]:
title.shape

In [ ]:
ing.shape

In [ ]:
ins.shape

In [ ]:
img_id

In [ ]:
find_img_path(img_id[0])